In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 


In [ ]:
#load the messy data
df = pd.read_excel('UK2024.J.xlsx')

In [ ]:
#create a table of only numbers 
df_numeric = df.select_dtypes(include=[np.number])

#check how many columns we have left
print(f"We have {df_numeric.shape[1]} numeric columns to analyse.")

In [ ]:
print(df.dtypes)

In [ ]:
#Mapping true=1 and false=0
#use .fillna(0) in case of empty cells

df['sme_flag'] = df['tender_suitability_sme'].map({True: 1, False: 0}).fillna(0)
df['vcse_flag'] = df['tender_suitability_vcse'].map({True: 1, False: 0}).fillna(0)

#converting accelerated column too
df['accelerated_flag'] = df['tender_procedure_isAccelerated'].map({True: 1, False: 0})

#finding lead time between pub and start
df['pub_date'] = pd.to_datetime(df['tender_datePublished'], errors='coerce', utc=True)
df['start_date'] = pd.to_datetime(df['tender_contractPeriod_startDate'], errors='coerce', utc=True)

#calc difference but force date
df['lead_time_days'] = (df['start_date'] - df['pub_date']).apply(lambda x: x.days)

df['lead_time_days'] = df['lead_time_days'].fillna(0)


#handle categories, avoiding 0, 1, 2 creating binary 'is_services' etc.
df = pd.concat([df, pd.get_dummies(df['tender_mainProcurementCategory'], prefix='cat')], axis=1)

#same for method, i.e same as above
df['proc_method_code'] = df['tender_procurementMethodDetails'].astype('category').cat.codes


#verifying new numeric count
new_numeric = df.select_dtypes(include=[np.number])
print(f"true, n= {new_numeric.shape[1]} numeric variables.")
print(new_numeric.columns.tolist())



In [ ]:
#fill remaining spaces with 0
X = new_numeric.fillna(0)

#1. Standardise, centre data so Mean=0 and std=1
scaler = StandardScaler()
X_Scaled = scaler.fit_transform(X)

#2. Use PCA to find all 6 principal components
pca = PCA()
pca_results = pca.fit_transform(X_Scaled)

#3. how much risk signal did we find
print("Explained variance ratio (eignvalues /Lambda):")
print(pca.explained_variance_ratio_)


In [ ]:
pc_labels = [f'PC{i+1}' for i in range(len(pca.explained_variance_ratio_))]
variances = pca.explained_variance_ratio_ * 100

plt.figure(figsize=(8, 5))
plt.bar(pc_labels, variances, color='green')
plt.ylabel('Percentage of Variance (%)')
plt.title('2024R.ArkData: Scree Plot')
plt.show()

In [ ]:
#table with PC1,2 and Status
pc_map = pd.DataFrame(pca_results[:, :2], columns=['PC1', 'PC2'])
pc_map['Status'] = df['tender_status'].values

#draw map
plt.figure(figsize=(12, 8))
sns.scatterplot(data=pc_map, x='PC1', y='PC2', hue='Status', alpha=0.5)
plt.title('Arkadian Risk Map: PC1 vs PC2')
plt.show()